In [27]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
path = "/Users/liuliangjun/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
path = "/Users/liuliangjun/.keras/models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = InceptionV3(weights=path, include_top=False, input_shape=(100, 100, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
# model.summary()

In [9]:
from glob import glob
from PIL import Image


train_set = '/Users/liuliangjun/PycharmProjects/vance/images/receipts/test/'

folder = '/Users/liuliangjun/PycharmProjects/vance/images/receipts/test/90/*'
exists = [i.split('/')[-1] for i in glob(folder)]

for path in glob(train_set+'0/*'):
    filename = path.split('/')[-1]
    if filename in exists:
        print(filename, 'exits.')
        continue
    img = Image.open(path)
    img_90 = img.transpose(Image.ROTATE_90)
    img_90.save(train_set+f'90/{filename}')
    img_180 = img.transpose(Image.ROTATE_180)
    img_180.save(train_set+f'180/{filename}')
    img_270 = img.transpose(Image.ROTATE_270)
    img_270.save(train_set+f'270/{filename}')


In [18]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = '/Users/liuliangjun/PycharmProjects/vance/images/receipts/train/'
test_dir = '/Users/liuliangjun/PycharmProjects/vance/images/receipts/test/'
train_datagen =  ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
)
test_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
)
train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size=(150, 150),
  batch_size=16,
)
validation_generator = test_datagen.flow_from_directory(
  test_dir,
  target_size=(150, 150),
  batch_size=16,
)

Found 1812 images belonging to 4 classes.
Found 376 images belonging to 4 classes.


In [34]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model.fit_generator(
  train_generator,
  steps_per_epoch=300,
  epochs=6,
  validation_data=validation_generator)

Epoch 1/6
300/300 [==============================] - 345s 1s/step - loss: 0.8408 - accuracy: 0.6595 - val_loss: 2.3713 - val_accuracy: 0.4628
Epoch 2/6
300/300 [==============================] - 347s 1s/step - loss: 0.7854 - accuracy: 0.7005 - val_loss: 0.7632 - val_accuracy: 0.4415
Epoch 3/6
300/300 [==============================] - 30929s 103s/step - loss: 0.7360 - accuracy: 0.7287 - val_loss: 0.6374 - val_accuracy: 0.5106
Epoch 4/6
300/300 [==============================] - 352s 1s/step - loss: 0.6601 - accuracy: 0.7588 - val_loss: 1.9839 - val_accuracy: 0.4920
Epoch 5/6
300/300 [==============================] - 349s 1s/step - loss: 0.6444 - accuracy: 0.7785 - val_loss: 1.6569 - val_accuracy: 0.5053
Epoch 6/6
300/300 [==============================] - 351s 1s/step - loss: 0.5904 - accuracy: 0.7944 - val_loss: 1.5869 - val_accuracy: 0.5186


In [28]:
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.models import Model
# from keras.applications.resnet import ResNet50
from keras.applications.resnet50 import ResNet50
# path = "/Users/liuliangjun/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# base_model = VGG16(weights=path, include_top=False, input_shape=(100, 100, 3))

# path = "/Users/liuliangjun/.keras/models/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
path = "/Users/liuliangjun/.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resnet50_weights_th_dim_ordering_th_kernels.h5
# base_model = InceptionV3(weights=path, include_top=False, input_shape=(100, 100, 3))
base_model = ResNet50(weights=path, include_top=False, input_shape=(150, 150, 3))
# base_model = VGG19(weights=path, include_top=False, input_shape=(100, 100, 3))

for layer in base_model.layers:
    layer.trainable = False
    
model = Flatten(name='flatten')(base_model.output)
model = Dense(128, activation='relu', name='fc1')(model)
model = Dense(128, activation='relu', name='fc2')(model)
# model = Dropout(0.5)(model)
model = Dense(4, activation='softmax')(model)
model = Model(inputs=base_model.input, outputs=model, name='inception')

# 打印模型结构，包括所需要的参数
model.summary()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________

In [31]:
# sparse_categorical_crossentropy   categorical_crossentropy
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_inbound_node',
 '_add_unique_metric_name',
 '_base_init',
 '_built',
 '_cache_output_metric_attributes',
 '_check_trainable_weights_consistency',
 '_collected_trainable_weights',
 '_compile_metric_functions',
 '_compile_metrics',
 '_compile_weighted_metrics',
 '_compute_previous_mask',
 '_expects_training_arg',
 '_feed_input_names',
 '_feed_input_shapes',
 '_feed_inputs',
 '_feed_loss_fns',
 '_feed_output_names',
 '_feed_output_shapes',
 '_feed_outputs',
 '_feed_sample_weight_modes',
 '_feed_sample_weights',
 '_feed_targets',
 '_function_kwargs',
 '_get_callback_model',

In [26]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image



def image_preporcess(img_path, target_size):
    # 以灰度图的形式读入图片
    image = cv2.imread(img_path)
    
    # resize 尺寸
    ih, iw = target_size
    # 原始图片尺寸
    h,  w, ns = image.shape

    # 计算缩放后图片尺寸
    scale = min(iw/w, ih/h)
    nw, nh = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))

    # 创建一张画布，画布的尺寸就是目标尺寸
    # fill_value=120为灰色画布
    image_paded = np.full(shape=[ih, iw], fill_value=120)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2

    # 将缩放后的图片放在画布中央
    image_paded[dh:nh+dh, dw:nw+dw] = image_resized
    
    # 归一化处理
    image_paded = image_paded / 255.

    return image_paded

if __name__=="__main__":
    # TODO cv2.imread(image_path, 0) 读取灰度图像
    image_path = "/Users/liuliangjun/PycharmProjects/vance/images/dogAndCat/train/cat.0.jpg"
    img=image_preporcess(image_path,(100,100))
    plt.imshow(img)

ValueError: could not broadcast input array from shape (74,100,3) into shape (74,100)

In [65]:
from glob import glob
from keras.preprocessing import image
image_path = '/Users/liuliangjun/PycharmProjects/vance/images/dogAndCat/train/*'

x = []
y = []
for i in glob(image_path):
    y.append(i.split('.')[0].split('/')[-1])
#     x.append(image_preporcess(i,(100,299)))
    x.append(image.img_to_array(image.load_img(i, target_size=(100, 100))))

x = np.array(x)
y = np.array(y)

In [66]:
y = np.array([1 if i =='dog' else 0 for i in y])

In [67]:
sum(y)

12500

In [97]:
# 训练将近一个小时

model.fit(x[:23000], y[:23000], batch_size=32, epochs=3, validation_data=(x[23000:], y[23000:]))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 742s 32ms/step - loss: 0.6061 - accuracy: 0.8329 - val_loss: 0.3428 - val_accuracy: 0.8915
Epoch 2/3
23000/23000 [==============================] - 710s 31ms/step - loss: 0.3042 - accuracy: 0.8875 - val_loss: 0.6949 - val_accuracy: 0.8625
Epoch 3/3
23000/23000 [==============================] - 954s 41ms/step - loss: 0.2605 - accuracy: 0.9095 - val_loss: 1.0191 - val_accuracy: 0.8840


In [103]:
# 训练将近一个小时

model.fit(x[:23000], y[:23000], batch_size=32, epochs=1, validation_data=(x[23000:], y[23000:]))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
10880/23000 [=============>................] - ETA: 14:45 - loss: 1.9632 - accuracy: 0.8436

KeyboardInterrupt: 

In [108]:
test_image_path = '/Users/liuliangjun/PycharmProjects/vance/images/dogAndCat/test/5.jpg'
# test_image_path = '/Users/liuliangjun/PycharmProjects/vance/images/dogAndCat/train/dog.1.jpg'
image_arr = image.img_to_array(image.load_img(test_image_path, target_size=(100, 100)))
# plt.imshow(image_arr)
# model.predict(np.expand_dims(image_arr, 0))
np.argmax(model.predict(np.expand_dims(image_arr, 0)),axis=1)

array([0])